In [ ]:
import numpy as np
import os
from glob import glob
import matplotlib.pyplot as plt
import nibabel as nib
from os.path import join as opj
from numpy import median, rint

from nipype.interfaces import spm
from nipype.interfaces import fsl
from nipype.interfaces.utility import Function, IdentityInterface
from nipype.interfaces.io import FreeSurferSource, SelectFiles, DataSink
from nipype.algorithms.misc import Gunzip
from nipype.algorithms.modelgen import SpecifySPMModel
from nipype.pipeline.engine import Workflow, Node, MapNode

# MATLAB - Specify path to current SPM and the MATLAB's default mode
from nipype.interfaces.matlab import MatlabCommand
MatlabCommand.set_default_paths('~/spm12/toolbox')
MatlabCommand.set_default_matlab_cmd("matlab -nodesktop -nosplash")
project_home = '/Users/myelin/Documents/asl_practice'

#other study-specific variables
subjects_dir = project_home + '/subs'
subjects_list = ['ko']
raw_dir = 'raw'
output_dir = 'proc'

#pre-proc variables
numslices = 38
TR = 4.844
asl_slice_order = range(1,numslices+1)
middle_slice = np.around(median(asl_slice_order), decimals=-1)
middle_slice = middle_slice.astype(int)

#Population specific variables for ASL
T1_gm = 1.2 #T1 of gray matter tissue in seconds (for CBF quantification)
nex_asl = 3 #number of excitations from the 3D ASL scan parameters
inversion_efficiency = 0.8 #from GE
background_supp_eff = 0.75 #from GE
efficiency = inversion_efficiency * background_supp_eff 
T1_blood = 1.65 #T1 of blood in seconds(1.6s at 3T and 1.4s at 1.5T)
sat_time = 2 #in seconds, from GE
partition_coeff = 0.9 #whole brain average in ml/g
scaling_factor = 32 #scaling factor, can be taken from PW dicom header
postlabel_delay = 1.525 #post label delay in seconds
labeling_time = 1.450 #labeling time in seconds